# 📘 Introduction to Pydantic

**Pydantic** is a powerful data validation library that uses Python type annotations to validate and parse data. It's widely used in modern Python applications, especially with FastAPI.

## 🎯 What You'll Learn

1. Creating data models with automatic validation
2. Type conversion and validation rules
3. Handling optional and default values
4. Working with nested structures
5. Custom validation logic
6. Advanced features for production applications

---

## 🚀 Section 1: Getting Started


### 📝 Creating Your First Model

A Pydantic model is a class that inherits from `BaseModel`. Each attribute is defined with a type annotation.


In [ ]:
from pydantic import BaseModel

class PersonCV(BaseModel):
    name: str
    address: str
    phone: str
    email: str


### 🔨 Creating Model Instances

There are three ways to create a Pydantic model instance:


In [ ]:
person_data = {
    "name": "Jack Smith",
    "address": "123 South West, New York, 17098",
    "phone": "1234567890",
    "email": "example@gmail.com"
}


**Method 1:** Using dictionary unpacking with `**`


In [ ]:
person_inst = PersonCV(**person_data)
person_inst


PersonCV(name='Jack Smith', address='123 South West, New York, 17098', phone='1234567890', email='example@gmail.com')

**Method 2:** Using `model_validate()` (recommended)


In [ ]:
person_inst = PersonCV.model_validate(person_data)
person_inst


PersonCV(name='Jack Smith', address='123 South West, New York, 17098', phone='1234567890', email='example@gmail.com')

**Method 3:** Using keyword arguments directly


In [ ]:
person_inst = PersonCV(
    name="Jack Smith",
    address="123 South West, New York, 17098",
    phone="1234567890",
    email="example@gmail.com"
)
person_inst


PersonCV(name='Jack Smith', address='123 South West, New York, 17098', phone='1234567890', email='example@gmail.com')

### 🔍 Accessing Model Data

You can access attributes directly or serialize to dictionaries and JSON.


In [ ]:
person_inst.name


'Jack Smith'

In [ ]:
person_inst.model_dump()


{'name': 'Jack Smith',
 'address': '123 South West, New York, 17098',
 'phone': '1234567890',
 'email': 'example@gmail.com'}

In [ ]:
person_inst.model_dump_json()


'{"name":"Jack Smith","address":"123 South West, New York, 17098","phone":"1234567890","email":"example@gmail.com"}'

---

## ✅ Section 2: Type Validation and Conversion

Pydantic automatically validates and converts data to the specified types.


### 🔢 Integer Validation

Pydantic will automatically convert string numbers to integers when possible.


In [ ]:
class PersonCV(BaseModel):
    name: str
    address: str
    phone: int
    email: str


In [ ]:
person_data = {
    "name": "Jack Smith",
    "address": "123 South West, New York, 17098",
    "phone": "1234567890",
    "email": "example@gmail.com"
}

PersonCV.model_validate(person_data)


PersonCV(name='Jack Smith', address='123 South West, New York, 17098', phone=1234567890, email='example@gmail.com')

### ✔️ Boolean Validation

Pydantic accepts multiple formats for boolean values: `True/False`, `1/0`, `"yes"/"no"`, `"true"/"false"`


In [ ]:
class PersonCV(BaseModel):
    name: str
    address: str
    phone: int
    email: str
    long_notice_period: bool


In [ ]:
person_data = {
    "name": "Jack Smith",
    "address": "123 South West, New York, 17098",
    "phone": "1234567890",
    "email": "example@gmail.com",
    "long_notice_period": True
}

PersonCV.model_validate(person_data)


PersonCV(name='Jack Smith', address='123 South West, New York, 17098', phone=1234567890, email='example@gmail.com', long_notice_period=True)

---

## 🔧 Section 3: Nullable and Default Values

Learn how to handle optional fields and provide default values.


### ❌ What Happens Without Nullable?

By default, `None` values are not accepted.


In [ ]:
from pydantic import ValidationError

class PersonCV(BaseModel):
    name: str
    address: str
    phone: int
    email: str
    long_notice_period: bool

person_data = {
    "name": "Jack Smith",
    "address": "123 South West, New York, 17098",
    "phone": "1234567890",
    "email": "example@gmail.com",
    "long_notice_period": None
}

try:
    PersonCV.model_validate(person_data)
except ValidationError as e:
    print("Model validation failed:", e)


### ✅ Making Fields Nullable

Use `bool | None` to allow `None` values.


In [ ]:
class PersonCV(BaseModel):
    name: str
    address: str
    phone: int
    email: str
    long_notice_period: bool | None

person_data = {
    "name": "Jack Smith",
    "address": "123 South West, New York, 17098",
    "phone": "1234567890",
    "email": "example@gmail.com",
    "long_notice_period": None
}

person_inst = PersonCV.model_validate(person_data)
person_inst


### 🎯 Setting Default Values

Fields with default values become optional.


In [ ]:
class PersonCV(BaseModel):
    name: str
    address: str
    phone: int
    email: str
    long_notice_period: bool | None = None

person_data = {
    "name": "Jack Smith",
    "address": "123 South West, New York, 17098",
    "phone": "1234567890",
    "email": "example@gmail.com"
}

PersonCV.model_validate(person_data)


---

## ⚙️ Section 4: Model Configuration

Customize model behavior and generate schemas.


### 📋 JSON Schema Generation

Pydantic can generate JSON schemas from your models, useful for API documentation.


In [ ]:
class PersonCV(BaseModel):
    name: str
    address: str
    phone: int
    email: str
    long_notice_period: bool | None = None

PersonCV.model_json_schema()


### 🚫 Handling Extra Fields

Control what happens when extra fields are provided.

- **`ignore`** (default): Extra fields are silently ignored
- **`forbid`**: Raises validation error if extra fields are present
- **`allow`**: Extra fields are stored in the model


In [ ]:
class PersonCV(BaseModel):
    name: str
    address: str
    phone: int
    email: str

person_data = {
    "name": "Jack Smith",
    "address": "123 South West, New York, 17098",
    "phone": "1234567890",
    "email": "example@gmail.com",
    "age": 45
}

PersonCV.model_validate(person_data)


In [ ]:
from pydantic import ConfigDict

class PersonCV(BaseModel):
    model_config = ConfigDict(extra='forbid')

    name: str
    address: str
    phone: int
    email: str

person_data = {
    "name": "Jack Smith",
    "address": "123 South West, New York, 17098",
    "phone": "1234567890",
    "email": "example@gmail.com",
    "age": 45
}

try:
    PersonCV.model_validate(person_data)
except ValidationError as e:
    print("Model validation failed:", e)


---

## 🏗️ Section 5: Nested Models

Real-world data often has hierarchical structures. Pydantic makes it easy to model nested data.


### 💼 Work Experience and Education

Create complex CV structures with nested models for work experience and education.


In [ ]:
class WorkExperience(BaseModel):
    company: str
    position: str
    years: int
    responsibilities: list[str] | None = None

class Education(BaseModel):
    institution: str
    degree: str
    year_graduated: int

class PersonCV(BaseModel):
    name: str
    email: str
    work_experience: list[WorkExperience]
    education: list[Education]

cv_data = {
    "name": "Jack Smith",
    "email": "jack.smith@example.com",
    "work_experience": [
        {
            "company": "Tech Corp",
            "position": "Senior Developer",
            "years": 3,
            "responsibilities": ["Code review", "Team leadership"]
        },
        {
            "company": "StartUp Inc",
            "position": "Developer",
            "years": 2
        }
    ],
    "education": [
        {
            "institution": "State University",
            "degree": "Bachelor in Computer Science",
            "year_graduated": 2015
        }
    ]
}

person_cv = PersonCV.model_validate(cv_data)
person_cv


---

## 📋 Section 6: Advanced Field Configuration

Use `Field` to add constraints, metadata, and documentation to your models.


### 🎯 Field Constraints and Metadata

The `Field` function allows you to:
- Add validation constraints (min/max length, ranges)
- Set default values and factories
- Add descriptions for documentation
- Define regex patterns


In [ ]:
from pydantic import Field

class PersonCV(BaseModel):
    name: str = Field(min_length=2, max_length=100, description="Full name of the candidate")
    email: str = Field(pattern=r"^[\w\.-]+@[\w\.-]+\.\w+$", description="Email address")
    age: int = Field(ge=18, le=100, description="Age must be between 18 and 100")
    years_experience: int = Field(0, ge=0, description="Years of professional experience")
    skills: list[str] = Field(default_factory=list, min_length=1, description="List of skills")

cv_data = {
    "name": "Jack Smith",
    "email": "jack.smith@example.com",
    "age": 30,
    "years_experience": 5,
    "skills": ["Python", "JavaScript", "SQL"]
}

person_cv = PersonCV.model_validate(cv_data)
person_cv


---

## 🔍 Section 7: Custom Validation

Implement custom business logic for data validation.


### ✏️ Field Validators

Apply custom validation logic to individual fields using the `@field_validator` decorator.


In [ ]:
from pydantic import field_validator

class PersonCV(BaseModel):
    name: str
    email: str
    phone: str

    @field_validator('name')
    @classmethod
    def name_must_contain_space(cls, v: str) -> str:
        if ' ' not in v:
            raise ValueError('Name must contain at least first and last name')
        return v.title()

    @field_validator('phone')
    @classmethod
    def phone_must_be_valid(cls, v: str) -> str:
        cleaned = v.replace('-', '').replace(' ', '').replace('(', '').replace(')', '')
        if not cleaned.isdigit() or len(cleaned) < 10:
            raise ValueError('Phone number must contain at least 10 digits')
        return cleaned

person_cv = PersonCV(
    name="john smith",
    email="john.smith@example.com",
    phone="(123) 456-7890"
)
person_cv


### 🔗 Model Validators

Validate multiple fields together using `@model_validator`.

- **`mode='before'`**: Validates raw input data before field validation
- **`mode='after'`**: Validates after all fields have been validated


In [ ]:
from typing import Any
from pydantic import model_validator

class PersonCV(BaseModel):
    name: str
    email: str
    years_experience: int
    age: int

    @model_validator(mode='before')
    @classmethod
    def check_sensitive_data_omitted(cls, data: Any) -> Any:
        if isinstance(data, dict):
            if 'ssn' in data:
                raise ValueError('SSN should not be included in CV')
            if 'salary_history' in data:
                raise ValueError('Salary history should not be included')
        return data

    @model_validator(mode='after')
    def check_experience_matches_age(self) -> 'PersonCV':
        if self.years_experience > (self.age - 18):
            raise ValueError('Years of experience cannot exceed age minus 18')
        return self

person_cv = PersonCV(
    name="Jack Smith",
    email="jack.smith@example.com",
    years_experience=5,
    age=25
)
person_cv


---

## 🚀 Section 8: Advanced Features

Explore powerful features for production applications.


### 🔄 Field Aliases

Map between different naming conventions (e.g., `camelCase` vs `snake_case`).


In [ ]:
class PersonCV(BaseModel):
    full_name: str = Field(alias='fullName')
    email_address: str = Field(alias='emailAddress')
    years_experience: int = Field(alias='yearsExperience')

cv_data = {
    "fullName": "Jack Smith",
    "emailAddress": "jack.smith@example.com",
    "yearsExperience": 5
}

person_cv = PersonCV.model_validate(cv_data)
person_cv


In [ ]:
person_cv.model_dump(by_alias=True)


### 🧮 Computed Fields

Derive values from existing fields without storing them explicitly.


In [ ]:
from pydantic import computed_field
from datetime import datetime

class PersonCV(BaseModel):
    name: str
    birth_year: int
    years_experience: int

    @computed_field
    @property
    def age(self) -> int:
        current_year = datetime.now().year
        return current_year - self.birth_year

    @computed_field
    @property
    def seniority_level(self) -> str:
        if self.years_experience < 2:
            return "Junior"
        elif self.years_experience < 5:
            return "Mid-level"
        else:
            return "Senior"

person_cv = PersonCV(
    name="Jack Smith",
    birth_year=1995,
    years_experience=6
)

person_cv.model_dump()


### 🔒 Strict Mode

Prevent automatic type coercion and require exact type matching.


In [ ]:
class PersonCV(BaseModel):
    name: str
    age: int
    years_experience: int

PersonCV.model_validate({"name": "Jack Smith", "age": "30", "years_experience": "5"})


In [ ]:
try:
    PersonCV.model_validate(
        {"name": "Jack Smith", "age": "30", "years_experience": "5"},
        strict=True
    )
except ValidationError as e:
    print("Validation Error:", e)


### 🔧 TypeAdapter

Validate non-BaseModel types like lists, dicts, or dataclasses.


In [ ]:
from pydantic import TypeAdapter

class PersonCV(BaseModel):
    name: str
    email: str

cv_list_adapter = TypeAdapter(list[PersonCV])

cv_data_list = [
    {"name": "Jack Smith", "email": "jack@example.com"},
    {"name": "Jane Doe", "email": "jane@example.com"}
]

validated_cvs = cv_list_adapter.validate_python(cv_data_list)
validated_cvs


In [ ]:
cv_list_adapter.json_schema()


---

## 🔐 Section 9: Configuration Management

Manage application settings with Pydantic Settings.


### ⚙️ Pydantic Settings

Load configuration from environment variables with automatic validation.


In [ ]:
import os
from pydantic_settings import BaseSettings

os.environ["CV_DATABASE_URL"] = "postgresql://localhost/cvs"
os.environ["CV_API_KEY"] = "secret-api-key-123"
os.environ["CV_MAX_UPLOAD_SIZE"] = "5"

class CVApplicationSettings(BaseSettings):
    database_url: str = Field(alias="cv_database_url")
    api_key: str = Field(alias="cv_api_key")
    max_upload_size_mb: int = Field(10, alias="cv_max_upload_size")
    debug_mode: bool = Field(False, alias="cv_debug_mode")

settings = CVApplicationSettings()
settings.model_dump()


### 🏷️ Environment Prefix

Use prefixes to group related settings and avoid naming conflicts.


In [ ]:
from pydantic_settings import SettingsConfigDict

os.environ["PRODUCTION_DATABASE_URL"] = "postgresql://prod-server/cvs"
os.environ["PRODUCTION_API_KEY"] = "prod-secret-key"
os.environ["PRODUCTION_MAX_UPLOAD_SIZE"] = "10"

class CVApplicationSettings(BaseSettings):
    model_config = SettingsConfigDict(env_prefix='production_')

    database_url: str
    api_key: str
    max_upload_size: int = 5

settings = CVApplicationSettings()
settings.model_dump()


---

## 📚 Summary

### ✨ Key Concepts Covered

1. **Basic Models**: Creating Pydantic models with type annotations
2. **Type Validation**: Automatic type conversion and validation (integers, booleans, strings)
3. **Nullable and Default Values**: Using `None` and default values for optional fields
4. **Model Configuration**: JSON schemas and extra field handling
5. **Nested Models**: Creating hierarchical data structures
6. **Field Configuration**: Constraints, metadata, and documentation
7. **Custom Validation**: Field validators and model validators
8. **Advanced Features**: Aliases, computed fields, specialized types, strict mode, TypeAdapter
9. **Settings Management**: Loading configuration from environment variables

### 💡 Best Practices

- ✅ Start with simple models and gradually add complexity
- ✅ Use type hints consistently for all fields
- ✅ Leverage built-in validators before writing custom ones
- ✅ Use computed fields for derived data instead of storing it
- ✅ Utilize Pydantic Settings for application configuration
- ✅ Always handle `ValidationError` exceptions in production code

### 🎯 Next Steps

- 🔹 Explore discriminated unions for polymorphic data
- 🔹 Learn about custom root validators
- 🔹 Integrate Pydantic with web frameworks (FastAPI, Django)
- 🔹 Use Pydantic for API request/response validation
- 🔹 Implement data serialization pipelines with Pydantic

---

### 🎓 Congratulations!

You now have a solid foundation in Pydantic. Practice by building your own data models and explore the [official documentation](https://docs.pydantic.dev/) for more advanced features.
